In [1]:
from tensorflow.python.keras import Input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import median_absolute_error, mean_absolute_percentage_error
from numpy import arange


In [5]:
# Read dataset:
dataset = pd.read_csv('Lab09_uncleaned_laptopData.csv')
print(f"There are {len(dataset.index)} instances.")
print(dataset.head())
plt.scatter(dataset['Inches'], dataset['Prices'])
plt.xlabel('Inches')
plt.ylabel('Prices')
plt.show()

There are 1303 instances.
   Unnamed: 0 Company   TypeName Inches                    ScreenResolution  \
0         0.0   Apple  Ultrabook   13.3  IPS Panel Retina Display 2560x1600   
1         1.0   Apple  Ultrabook   13.3                            1440x900   
2         2.0      HP   Notebook   15.6                   Full HD 1920x1080   
3         3.0   Apple  Ultrabook   15.4  IPS Panel Retina Display 2880x1800   
4         4.0   Apple  Ultrabook   13.3  IPS Panel Retina Display 2560x1600   

                          Cpu   Ram               Memory  \
0        Intel Core i5 2.3GHz   8GB            128GB SSD   
1        Intel Core i5 1.8GHz   8GB  128GB Flash Storage   
2  Intel Core i5 7200U 2.5GHz   8GB            256GB SSD   
3        Intel Core i7 2.7GHz  16GB            512GB SSD   
4        Intel Core i5 3.1GHz   8GB            256GB SSD   

                            Gpu  OpSys  Weight        Price  
0  Intel Iris Plus Graphics 640  macOS  1.37kg   71378.6832  
1        Intel

KeyError: 'Prices'

In [ ]:
def split_dataset(dataset, train_frac=0.7):
    train = dataset.sample(frac=train_frac)
    val = dataset.drop(train.index)
    return train, val

# Split dataset into train and validation:
train, validation = split_dataset(dataset, train_frac=0.7)
plt.scatter(train['x'], train['y'], c='blue', alpha=0.4)
plt.scatter(validation['x'], validation['y'], c='red', alpha=0.4)
plt.xlabel('x')
plt.ylabel('y')
plt.legend(['train set', 'validation set'], framealpha=0.3)
plt.show()


In [13]:
# # Create model structure
# from tensorflow.python.keras.models import Sequential
# from tensorflow.python.keras import Input
# from tensorflow.python.keras.layers import Dense

In [14]:
# # Create model:
# model = Sequential()
# model.add(Input(shape=(1,)))
# model.add(Dense(200, activation='sigmoid'))
# model.add(Dense(200, activation='sigmoid'))
# model.add(Dense(200, activation='sigmoid'))
# model.add(Dense(1, activation='linear'))
# print(model.summary())


In [15]:
import tensorflow as tf
import keras
from keras import layers, models

In [ ]:
# create model
model = keras.Sequential()
model.add(keras.layers.InputLayer(shape=(1,)))
model.add(keras.layers.Dense(200, activation='sigmoid'))
model.add(keras.layers.Dense(200, activation='sigmoid'))
model.add(keras.layers.Dense(200, activation='sigmoid'))
model.add(keras.layers.Dense(1, activation='linear'))
print(model.summary())

In [ ]:
# Train:
loss = 'mse'
metric = 'mae'

epochs = 1750

# dataset is split ‘X’ as input to model
# ‘Y’ as the expected output of model 
x_train, y_train = train['x'], train['y']
x_val, y_val = validation['x'], validation['y']
model.compile(loss=loss, optimizer='adam', metrics=[metric])
history = model.fit(x_train, y_train, epochs=epochs, batch_size=64, verbose=1, validation_data=(x_val, y_val))


In [ ]:
# Display loss:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()
# Display metric:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.title('model mean absolute error (mae)')
plt.ylabel('mae')
plt.xlabel('epoch')
plt.legend(['train', 'test'])
plt.show()
